In [5]:
#load modules

In [7]:
from GraphSL.GNN.SLVAE.main import SLVAE
from GraphSL.GNN.IVGD.main import IVGD
from GraphSL.GNN.GCNSI.main import GCNSI
from GraphSL.Prescribed import LPSI, NetSleuth, OJC
from GraphSL.utils import load_dataset, diffusion_generation, split_dataset
import os

In [11]:
curr_dir = os.getcwd()
print (curr_dir)

/home/dsss/junwang/personal/GraphSL


In [12]:
# load datasets ('karate', 'dolphins', 'jazz', 'netscience', 'cora_ml', 'power_grid')
data_name = 'karate'
graph = load_dataset(data_name, data_dir=curr_dir)
print(graph)

{'adj_mat': <34x34 sparse matrix of type '<class 'numpy.float32'>'
	with 156 stored elements in Compressed Sparse Row format>}


In [13]:
# generate diffusion
dataset = diffusion_generation(graph=graph, infect_prob=0.3, diff_type='IC', sim_num=100, seed_ratio=0.2)
print(dataset)

{'adj_mat': <34x34 sparse matrix of type '<class 'numpy.float32'>'
	with 156 stored elements in Compressed Sparse Row format>, 'diff_mat': tensor([[[0., 1.],
         [1., 1.],
         [1., 1.],
         ...,
         [1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.],
         ...,
         [0., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [0., 1.],
         [0., 1.],
         ...,
         [1., 1.],
         [1., 1.],
         [1., 1.]],

        ...,

        [[0., 1.],
         [1., 1.],
         [1., 1.],
         ...,
         [1., 1.],
         [0., 1.],
         [1., 1.]],

        [[0., 1.],
         [0., 1.],
         [1., 1.],
         ...,
         [1., 1.],
         [0., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.],
         ...,
         [1., 1.],
         [0., 1.],
         [0., 1.]]])}


In [16]:
# split into training and test sets
adj, train_dataset, test_dataset = split_dataset(dataset)
print(f"adj={adj}")
print(train_dataset)
print(test_dataset)

adj=  (0, 1)	1.0
  (0, 2)	1.0
  (0, 3)	1.0
  (0, 4)	1.0
  (0, 5)	1.0
  (0, 6)	1.0
  (0, 7)	1.0
  (0, 8)	1.0
  (0, 10)	1.0
  (0, 11)	1.0
  (0, 12)	1.0
  (0, 13)	1.0
  (0, 17)	1.0
  (0, 19)	1.0
  (0, 21)	1.0
  (0, 31)	1.0
  (1, 0)	1.0
  (1, 2)	1.0
  (1, 3)	1.0
  (1, 7)	1.0
  (1, 13)	1.0
  (1, 17)	1.0
  (1, 19)	1.0
  (1, 21)	1.0
  (1, 30)	1.0
  :	:
  (32, 18)	1.0
  (32, 20)	1.0
  (32, 22)	1.0
  (32, 23)	1.0
  (32, 29)	1.0
  (32, 30)	1.0
  (32, 31)	1.0
  (32, 33)	1.0
  (33, 8)	1.0
  (33, 9)	1.0
  (33, 13)	1.0
  (33, 14)	1.0
  (33, 15)	1.0
  (33, 18)	1.0
  (33, 19)	1.0
  (33, 20)	1.0
  (33, 22)	1.0
  (33, 23)	1.0
  (33, 26)	1.0
  (33, 27)	1.0
  (33, 28)	1.0
  (33, 29)	1.0
  (33, 30)	1.0
  (33, 31)	1.0
  (33, 32)	1.0


In [17]:
# LPSI
print("LPSI:")
lpsi = LPSI()

# train LPSI
alpha, thres, auc, f1, pred = lpsi.train(adj, train_dataset)
print(f"train auc: {auc:.3f}, train f1: {f1:.3f}")

# test LPSI
metric = lpsi.test(adj, test_dataset, alpha, thres)
print(f"test acc: {metric.acc:.3f}, test pr: {metric.pr:.3f}, test re: {metric.re:.3f}, test f1: {metric.f1:.3f}, test auc: {metric.auc:.3f}")

LPSI:
alpha = 0.001, train_auc = 0.323
alpha = 0.01, train_auc = 0.323
alpha = 0.1, train_auc = 0.323
thres = 0.100
thres = 0.100, train_f1 = 0.353
thres = 0.300
thres = 0.300, train_f1 = 0.353
thres = 0.500
thres = 0.500, train_f1 = 0.353
thres = 0.700
thres = 0.700, train_f1 = 0.353
thres = 0.900
thres = 0.900, train_f1 = 0.353
train auc: 0.323, train f1: 0.353
test acc: 0.353, test pr: 0.214, test re: 1.000, test f1: 0.353, test auc: 0.320


In [18]:
# NetSleuth
print("NetSleuth:")
netSleuth = NetSleuth()

# train NetSleuth
k, auc, f1 = netSleuth.train(adj, train_dataset)
print(f"train auc: {auc:.3f}, train f1: {f1:.3f}")

# test NetSleuth
metric = netSleuth.test(adj, test_dataset, k)
print(f"test acc: {metric.acc:.3f}, test pr: {metric.pr:.3f}, test re: {metric.re:.3f}, test f1: {metric.f1:.3f}, test auc: {metric.auc:.3f}")

NetSleuth:
k = 5, train_auc = 0.583
k = 10, train_auc = 0.684
train auc: 0.684, train f1: 0.448
test acc: 0.741, test pr: 0.360, test re: 0.600, test f1: 0.450, test auc: 0.686


In [19]:
# OJC
print("OJC:")
ojc = OJC()

# train OJC
Y, auc, f1 = ojc.train(adj, train_dataset)
print(f"train auc: {auc:.3f}, train f1: {f1:.3f}")

# test OJC
metric = ojc.test(adj, test_dataset, Y)
print(f"test acc: {metric.acc:.3f}, test pr: {metric.pr:.3f}, test re: {metric.re:.3f}, test f1: {metric.f1:.3f}, test auc: {metric.auc:.3f}")

OJC:
Y = 5, train_auc = 0.422
Y = 10, train_auc = 0.422
train auc: 0.422, train f1: 0.379
test acc: 0.715, test pr: 0.315, test re: 0.525, test f1: 0.394, test auc: 0.640


In [20]:
# GCNSI
print("GCNSI:")
gcnsi = GCNSI()

# train GCNSI
gcnsi_model, thres, auc, f1, pred = gcnsi.train(adj, train_dataset)
print(f"train auc: {auc:.3f}, train f1: {f1:.3f}")

# test GCNSI
metric = gcnsi.test(adj, test_dataset, gcnsi_model, thres)
print(f"test acc: {metric.acc:.3f}, test pr: {metric.pr:.3f}, test re: {metric.re:.3f}, test f1: {metric.f1:.3f}, test auc: {metric.auc:.3f}")

GCNSI:
epoch = 0, loss = 7.355
epoch = 10, loss = 0.815
epoch = 20, loss = 0.721
epoch = 30, loss = 0.663
epoch = 40, loss = 0.742
epoch = 50, loss = 0.620
epoch = 60, loss = 0.619
epoch = 70, loss = 0.632
epoch = 80, loss = 0.606
epoch = 90, loss = 0.624
train_auc = 0.931
thres = 0.100, train_f1 = 0.545
thres = 0.300, train_f1 = 0.493
thres = 0.500, train_f1 = 0.317
thres = 0.700, train_f1 = 0.152
thres = 0.900, train_f1 = 0.000
train auc: 0.931, train f1: 0.545
test acc: 0.706, test pr: 0.375, test re: 1.000, test f1: 0.545, test auc: 0.931


In [21]:
# IVGD
print("IVGD:")
ivgd = IVGD()

# train IVGD diffusion
diffusion_model = ivgd.train_diffusion(adj, train_dataset)

# train IVGD
ivgd_model, thres, auc, f1, pred = ivgd.train(
    adj, train_dataset, diffusion_model)
print(f"train auc: {auc:.3f}, train f1: {f1:.3f}")

# test IVGD
metric = ivgd.test(test_dataset, diffusion_model, ivgd_model, thres)
print(f"test acc: {metric.acc:.3f}, test pr: {metric.pr:.3f}, test re: {metric.re:.3f}, test f1: {metric.f1:.3f}, test auc: {metric.auc:.3f}")

IVGD:
Epoch 0: Train loss = 0.5690, Train error = 0.5690, early stopping loss = 0.6661, early stopping error = 0.6661, (1.052 sec)
Epoch 10: Train loss = 0.5370, Train error = 0.5370, early stopping loss = 0.4158, early stopping error = 0.4158, (6.168 sec)
Epoch 20: Train loss = 0.4996, Train error = 0.4996, early stopping loss = 0.4988, early stopping error = 0.4988, (6.162 sec)
Epoch 30: Train loss = 0.4909, Train error = 0.4909, early stopping loss = 0.5423, early stopping error = 0.5423, (6.164 sec)
Epoch 40: Train loss = 0.4839, Train error = 0.4839, early stopping loss = 0.5152, early stopping error = 0.5152, (6.175 sec)
Epoch 50: Train loss = 0.5062, Train error = 0.5062, early stopping loss = 0.5135, early stopping error = 0.5135, (6.198 sec)
train mean error:0.492
early_stopping mean error:0.530
validation mean error:0.499
run time:32.534 seconds
run time per epoch:0.626 seconds
epoch 0: loss = 0.452
epoch 10: loss = 0.428
epoch 20: loss = 0.426
epoch 30: loss = 0.418
epoch 40

In [22]:
# SLVAE
print("SLVAE:")
slave = SLVAE()

# train SLVAE
slvae_model, seed_vae_train, thres, auc, f1, pred = slave.train(
    adj, train_dataset)
print(f"train auc: {auc:.3f}, train f1: {f1:.3f}")

# test SLVAE
metric = slave.infer(test_dataset, slvae_model, seed_vae_train, thres)
print(f"test acc: {metric.acc:.3f}, test pr: {metric.pr:.3f}, test re: {metric.re:.3f}, test f1: {metric.f1:.3f}, test auc: {metric.auc:.3f}")

SLVAE:
train SLVAE:
epoch = 0, loss = 0.719
epoch = 10, loss = 0.294
epoch = 20, loss = 0.224
epoch = 30, loss = 0.171
epoch = 40, loss = 0.168
epoch = 50, loss = 0.346
epoch = 60, loss = 0.162
epoch = 70, loss = 0.156
epoch = 80, loss = 0.190
epoch = 90, loss = 0.150
infer seed from training set:
epoch = 0, obj = -3.8282
epoch = 2, obj = -3.8273
epoch = 4, obj = -3.8219
epoch = 6, obj = -3.8306
epoch = 8, obj = -3.8224
epoch = 10, obj = -3.8184
epoch = 12, obj = -3.8212
epoch = 14, obj = -3.8229
epoch = 16, obj = -3.8164
epoch = 18, obj = -3.8190
thres = 0.100, train_f1 = 0.543
thres = 0.300, train_f1 = 0.230
thres = 0.500, train_f1 = 0.098
thres = 0.700, train_f1 = 0.038
thres = 0.900, train_f1 = 0.010
train auc: 0.937, train f1: 0.543
infer seed from test set:
epoch = 0, obj = -3.8479
epoch = 1, obj = -3.8509
epoch = 2, obj = -3.8494
epoch = 3, obj = -3.8514
epoch = 4, obj = -3.8449
epoch = 5, obj = -3.8505
epoch = 6, obj = -3.8417
epoch = 7, obj = -3.8575
epoch = 8, obj = -3.8545
e